# Wrocław Public Transport

In this notebook I analyze performance of Wrocław's public transport from 2022-04-13 to 2022-04-30.

## Data Loading and Cleaning

In [1]:
import numpy as np
import math
import polars as pl
import requests

from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.palettes import Inferno256
from bokeh.plotting import figure, show
from bokeh.tile_providers import OSM, get_provider
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin, cartesian_to_axial

tile_provider = get_provider(OSM)
color_mapper = LinearColorMapper(palette=Inferno256)
output_notebook()

Loading BokehJS ...

In [2]:
positions = pl.read_csv("positions.csv")

In [3]:
# https://wiki.openstreetmap.org/wiki/Mercator#Python
# drop duplicates and invalid data, coordinate/time conversions
MERCATOR_RADIUS = 6378137

positions = positions.unique(
    subset=["k", "timestamp"]
).filter(
    (pl.col("x") > 50.5) & (pl.col("x") < 51.5) & (pl.col("y") > 16.5) & (pl.col("y") < 17.5)
).sort(
    ["k", "timestamp"]
).with_column(
    pl.lit(2).alias("offset")
).with_columns([
    # necessary workaround with offset as polars does not support time zones
    ((pl.col("timestamp")+"Z").str.strptime(pl.Datetime, fmt="%+", strict=False) + pl.duration(hours="offset")).alias("timestamp"),
    np.deg2rad(pl.col("x")).alias("x_rad"),
    np.deg2rad(pl.col("y")).alias("y_rad"),
]).with_columns([
    pl.internals.expr.ExprDateTimeNameSpace.seconds(pl.col("timestamp")-pl.col("timestamp").shift(1)).alias("time_diff_s"),
    ((pl.col("x_rad")/2 + math.pi/4).tan().log() * MERCATOR_RADIUS).alias("y_mercator"),
    (pl.col("y_rad") * MERCATOR_RADIUS).alias("x_mercator"),
])

In [4]:
# https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
positions['distance_km'] = positions.select([
    pl.col("x_rad").alias("x"),
    pl.col("y_rad").alias("y"),
    pl.col("x_rad").shift(1).alias("prev_x"),
    pl.col("y_rad").shift(1).alias("prev_y")
]).with_columns([
    (pl.col("x") - pl.col("prev_x")).alias("d_lat"),
    (pl.col("y") - pl.col("prev_y")).alias("d_lon"),
]).with_columns([
    ((pl.col("d_lat") / 2).sin().pow(2) + (pl.col("d_lon") / 2).sin().pow(2) * pl.col("prev_x").cos() * pl.col("x").cos()).alias("a")
]).with_columns([
    ((pl.col("a").sqrt() / (1-pl.col("a")).sqrt()).arctan() * 2 * 6373).alias("distance_km")
])['distance_km']

In [5]:
# k is not unique across full data, this makes it one ride identifier
RIDE_END = 3600
positions = positions.with_column(
    (pl.col("k")*10).alias("k")
).with_column(
    (pl.col("time_diff_s") > RIDE_END).cumsum().fill_null("backward").over("k").alias("ride_id_suffix")
).with_column(
    (pl.col("k")+pl.col("ride_id_suffix")).alias("k")
)

In [6]:
positions = positions.with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("time_diff_s")).alias("time_diff_s")
).with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("distance_km")).alias("distance_km")
).with_column(
    (pl.col("distance_km") / pl.col("time_diff_s") * 3600).alias("speed_km_h")
)

In [7]:
# Sometimes vehicles have GPS active when they finished their drive and stand idle.
# This needs to be cleaned in order to have correct data about drive times.

MIN_OBS = 40
CUTOFF_KM_H = 5

positions = positions.filter(
    (pl.col("speed_km_h").max().over("k") > 5) # max speed over 5km/h
).with_columns([
    (pl.col("speed_km_h") > CUTOFF_KM_H).cumsum().fill_null("backward").over("k").alias("cutoff"),
]).filter(
    pl.col("cutoff").is_between(0, pl.col("cutoff").max().over("k"))
).filter(
    pl.col("k").count().over("k") >= MIN_OBS
)

In [8]:
rides = positions.groupby('k').agg([
    pl.col("name").first().alias("name"),
    pl.col("timestamp").first().alias("start_time"),
    (pl.col("distance_km").sum() / pl.col("time_diff_s").sum() * 3600).alias("speed_km_h_avg")
]).sort("start_time")

## Fastest Lines

- 2xx are night buses
- 1xx are regular buses
- single/two digit are trams
- letters are "fast" buses

In [9]:
line_stats = rides.groupby("name").agg([
    pl.col("k").n_unique().alias("rides"),
    pl.col("speed_km_h_avg").median().alias("speed_km_h_avg_median"),
]).sort("speed_km_h_avg_median")
source = ColumnDataSource(data=line_stats.to_dict(as_series=False))

tooltips=[
    ("Line", "@name"),
    ("Unique rides", "@rides"),
    ("Median ride speed", "@speed_km_h_avg_median"),
]

p = figure(
    title="Median ride speed by line",
    tools="",
    toolbar_location=None,
    y_range=line_stats["name"].to_list(),
    x_range=[0, 50],
    tooltips=tooltips,
    height=1800
)
p.segment(source=source, x0=0, y0="name", x1="speed_km_h_avg_median", y1="name", line_width=2, line_color="green", )
p.circle(source=source, x="speed_km_h_avg_median", y="name", size=15, fill_color="orange", line_color="green", line_width=3, )

show(p)

## Performance by time of the day

Note that 17-18 April were Easter holidays

In [10]:
rides_by_time = rides.groupby_dynamic(index_column="start_time", every="30m").agg([
    pl.median("speed_km_h_avg").alias("median_avg_speed"),
    pl.count("k").alias("rides"),
]).with_column(
    pl.col("start_time").cast(pl.Utf8).alias("tooltip_date")
).sort("start_time")

rides_by_time = ColumnDataSource(data=rides_by_time.to_dict(as_series=False))

In [11]:
tooltips=[
    ("Start time", "@tooltip_date"),
    ("Rides", "@rides"),
]

p1 = figure(
    title="Rides by time of the day",
    x_axis_type = "datetime",
    x_axis_label='Time',
    y_axis_label='Rides',
    tooltips=tooltips,
    width=900,
    height=400
)
p1.line(
    source=rides_by_time,
    x="start_time",
    y="rides",
    line_width=2
)

show(p1)

In [12]:
tooltips=[
    ("Start time", "@tooltip_date"),
    ("Median ride speed", "@median_avg_speed"),
]

p2 = figure(
    title="Speed by time of the day",
    x_axis_type = "datetime",
    x_axis_label='Time',
    y_axis_label='Ride speed',
    tooltips=tooltips,
    width=900,
    height=400
)
p2.line(
    source=rides_by_time,
    x="start_time",
    y="median_avg_speed",
    line_width=2
)
show(p2)

## Sample Tram Ride

In [13]:
sample_ride = positions.filter(pl.col("k") == 197090560)
source = ColumnDataSource(
    data=sample_ride.select([
        pl.col("x_mercator"),
        pl.col("y_mercator"),
        pl.col("timestamp"),
    ]).to_dict(as_series=False)
)

p = figure(
    title="Sample ride: Tram 9",
    x_range=(1875733.4, 1914695.2), 
    y_range=(6621293.7, 6674532.7),
    x_axis_type="mercator",
    y_axis_type="mercator"
)
p.add_tile(tile_provider)
p.circle(
    source=source,
    x="x_mercator",
    y="y_mercator",
    size=10,
    fill_color={"field": "timestamp", "transform": color_mapper},
    fill_alpha=0.8
)
show(p)


## Vehicle positions

In [14]:
# https://github.com/bokeh/bokeh/blob/664ead5306bba64609e734d4105c8aa8cfb76d81/bokeh/util/hex.py#L101
# workaround necessary since hexbin assumes pandas
q, r = cartesian_to_axial(
    x=positions["x_mercator"].to_numpy(),
    y=positions["y_mercator"].to_numpy(),
    size=400,
    orientation="pointytop",
    aspect_scale=1.0
)
bins = positions.with_columns([
    pl.lit(q).alias("q"),
    pl.lit(r).alias("r")
]).groupby(
    ["q", "r", ]
).agg([
    pl.col("k").count().alias("count"),
    pl.col("speed_km_h").median().alias("median_speed"),
]).with_columns([
    (pl.col('count')+1).log().alias("count_log"),
    (pl.col('median_speed')+1).log().alias("median_speed_log"),
])
bins = ColumnDataSource(data=bins.to_dict(as_series=False))

In [15]:
p = figure(
    title="Most common vehicle locations",
    x_range=(1875733.4, 1914695.2), 
    y_range=(6621293.7, 6674532.7),
    x_axis_type="mercator",
    y_axis_type="mercator"
)
p.add_tile(tile_provider)

p.hex_tile(
    q="q",
    r="r",
    size=400,
    line_color=None,
    alpha=0.6,
    source=bins,
    fill_color=linear_cmap('count', 'Inferno256', 0, max(bins.data['count']))
)
show(p)

In [16]:
p = figure(
    title="Median vehicle speed by location (Log scale)",
    x_range=(1875733.4, 1914695.2), 
    y_range=(6621293.7, 6674532.7),
    x_axis_type="mercator",
    y_axis_type="mercator"
)
p.add_tile(tile_provider)

p.hex_tile(
    q="q",
    r="r",
    size=400,
    line_color=None,
    alpha=0.6,
    source=bins,
    fill_color=linear_cmap('median_speed_log', 'Inferno256', 0, max(bins.data['median_speed_log']))
)
show(p)